In [3]:
import tensorflow as tf
import tensorflow.feature_column as fc
import csv
import sys
import os
import pandas as pd
import numpy as np

In [34]:
LAB_PATH=os.path.abspath(os.curdir)
DATASET_PATH=os.path.join(LAB_PATH,"datasets")
file_path=os.path.join(DATASET_PATH,"ds2.csv")
BATCH_SIZE=32
STEPS=1000
NUM_EPOCHS=40
data = pd.read_csv(file_path)
print("before",data)
x_norm_factor=data["Input"].max()-data["Input"].min()
y_norm_factor=data["Output"].max()-data["Output"].min()
print(x_norm_factor,y_norm_factor)
data=data/(data.max()-data.min())
print("after",data)

before            Input       Output
0    -194.350326  -317.830587
1     924.174084  1695.513351
2    -241.935738  -403.484328
3     103.498579   218.297443
4     -16.816085     1.731047
5     783.198644  1441.757559
6      98.224513   208.804124
7     -80.662302  -113.192145
8     281.523912   538.743041
9     558.045288  1036.481518
10   -160.245277  -256.441499
11    181.908714   359.435685
12    477.231942   891.017495
13    856.625121  1573.925218
14     66.024884   150.844792
15     33.842973    92.917351
16    -91.893956  -133.409121
17    548.830930  1019.895674
18      3.175077    37.715138
19    576.842068  1070.315722
20    345.692290   654.246122
21    399.098010   750.376419
22    -59.790237   -75.622427
23    -13.284477     8.087941
24    503.812294   938.862130
25    614.157709  1137.483876
26    754.975079  1390.955142
27    514.038480   957.269264
28    534.978478   994.961261
29    599.328696  1110.791653
...          ...          ...
9970 -103.141892  -153.655406
997

In [35]:
# get the two dataset columns as numpy arrays

def get_input():
    # get the two dataset columns as numpy arrays
    # normalize all the values
    #data[['Input','Output']] /= data[['Input','Output']].max()
    global data
    #data=(data-data.mean())/data.std()
    #print("after",data)
    train_x = data['Input'].values
    train_y = data['Output'].values

    dataset = tf.data.Dataset.from_tensor_slices(({'Input':train_x}, train_y))

    # apply transformations to the dataset
    dataset = dataset.batch(BATCH_SIZE)  # get the data in batches of BATCH_SIZE
    dataset = dataset.repeat(NUM_EPOCHS) # need enough rows for training
    return dataset


In [36]:
# Estimator using the default optimizer.
def train_lin_reg():
    estimator = tf.estimator.LinearRegressor(feature_columns=[fc.numeric_column('Input')])#,hidden_units = [128,64,32,16])
    estimator.train(input_fn = get_input)
    metrics=estimator.evaluate(input_fn=get_input)
    return estimator
    

In [37]:
est=train_lin_reg()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_session_config': None, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_task_type': 'worker', '_service': None, '_num_worker_replicas': 1, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000272C2AF1438>, '_is_chief': True, '_model_dir': 'C:\\Users\\Admin\\AppData\\Local\\Temp\\tmpwtuv_nqz', '_train_distribute': None, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_device_fn': None, '_evaluation_master': '', '_master': '', '_global_id_in_cluster': 0}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 int

INFO:tensorflow:loss = 3.0132147e-14, step = 6901 (0.086 sec)
INFO:tensorflow:global_step/sec: 1143.51
INFO:tensorflow:loss = 5.782549e-14, step = 7001 (0.084 sec)
INFO:tensorflow:global_step/sec: 1108.29
INFO:tensorflow:loss = 1.07372444e-13, step = 7101 (0.090 sec)
INFO:tensorflow:global_step/sec: 1115.39
INFO:tensorflow:loss = 5.3524025e-14, step = 7201 (0.093 sec)
INFO:tensorflow:global_step/sec: 1048.18
INFO:tensorflow:loss = 4.8575727e-14, step = 7301 (0.092 sec)
INFO:tensorflow:global_step/sec: 1143.83
INFO:tensorflow:loss = 5.0199428e-14, step = 7401 (0.087 sec)
INFO:tensorflow:global_step/sec: 1055.49
INFO:tensorflow:loss = 5.719123e-14, step = 7501 (0.095 sec)
INFO:tensorflow:global_step/sec: 1102.62
INFO:tensorflow:loss = 7.3427375e-14, step = 7601 (0.090 sec)
INFO:tensorflow:global_step/sec: 1121.15
INFO:tensorflow:loss = 5.713485e-14, step = 7701 (0.096 sec)
INFO:tensorflow:global_step/sec: 1034.22
INFO:tensorflow:loss = 4.3153198e-14, step = 7801 (0.093 sec)
INFO:tensorfl

In [38]:
def denormalise(pred,bias,ele):
    pred*=y_norm_factor
    bias*=y_norm_factor
    ele*=x_norm_factor
    return pred,bias,ele



def get_equation(est):
    global data
    train_x = data['Input'].values[:10]  # as a numpy array
    train_y = data['Output'].values[:10]
    ele=train_x[1]#its normalised as here!
    print(train_x[1],train_y[1])
    #est = tf.estimator.LinearRegressor(feature_columns=[fc.numeric_column('Input')])
    #bias=list(estimator.predict(input_fn = get_input))
    zero_inp=tf.estimator.inputs.numpy_input_fn(x = {'Input':np.array([0.0],dtype = np.float64)}, shuffle = False)
    rand_inp=tf.estimator.inputs.numpy_input_fn(x = {'Input':np.array([ele],dtype = np.float64)}, shuffle = False)
    bias=list(est.predict(zero_inp))[0]['predictions'][0]
    pred=list(est.predict(rand_inp))[0]['predictions'][0]
    pred,bias,ele=denormalise(pred,bias,ele)
    slope=(pred-bias)/ele
    print("bias is ",bias,"pred is",pred,"slope is",slope)
    print("The equation is-->","y = ",slope,"x + ",bias)

    
#F = 1.8*C + 32 
get_equation(est)

0.7394369261311728 0.7536610270869751
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Admin\AppData\Local\Temp\tmpwtuv_nqz\model.ckpt-12520
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Admin\AppData\Local\Temp\tmpwtuv_nqz\model.ckpt-12520
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
bias is  32.00005872379652 pred is 1695.5132380644095 slope is 1.7999998144295648
The equation is--> y =  1.7999998144295648 x +  32.00005872379652
